In [3]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.3/216.3 MB 4.0 MB/s eta 0:00:0000:0100:02
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      Traceback (most recent call last):
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks

In [8]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      Traceback (most recent call last):
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pkg_resources/_vendor/packagin

In [11]:
import tensorflow as tf
import os

2023-08-05 13:15:05.687533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
import cv2
import imghdr

In [13]:
data_dir = 'Cars'
correct_ext = ['jpeg', 'jpg', 'bmp', 'png']

NameError: name 'imghdr' is not defined

In [20]:
for brand in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, brand)):
        image_path = os.path.join(data_dir, brand, image)
        try:
            img = cv2.imread(image_path)
            img_ext = imghdr.what(image_path)
            if img_ext not in correct_ext:
                print(f'Image does not have the correct extension: {image_path}')
                os.remove(image_path)
        except Exception as e:
            print(e)
            print(f"Issue with image {image_path}")

Image does not have the correct extension: Cars/Volkswagen/Volkswagen-Tiguan-2017-all-space-ext-1a-front.jpg
Image does not have the correct extension: Cars/Volkswagen/12666462165_bb634ac64d_c.jpg
Image does not have the correct extension: Cars/Volkswagen/2023-volkswagen-id.-buzz-rendering.webp
Image does not have the correct extension: Cars/Volkswagen/1920x1080.webp
Image does not have the correct extension: Cars/Volkswagen/1200px-2007_Volkswagen_Golf_281K_MY0729_Sportline_2.0_TDI_5-door_hatchback_282010-07-0529.jpg
Image does not have the correct extension: Cars/Volkswagen/.DS_Store
Image does not have the correct extension: Cars/Volkswagen/25vwtaigo.jpg
Image does not have the correct extension: Cars/Volkswagen/VOLKSWAGEN-GOLF-00.webp
Image does not have the correct extension: Cars/Volkswagen/2017-volkswagen-beetle.webp
Image does not have the correct extension: Cars/Volkswagen/stationwagen_volkswagen_variant_1_dot_2_tsi_comf_exe_nav_2012_7395397-12.jpg
Image does not have the corre

libpng warning: iCCP: known incorrect sRGB profile


Image does not have the correct extension: Cars/Porsche/1920x1080 2.webp
Image does not have the correct extension: Cars/Porsche/PORSCHE-911-003.webp
Image does not have the correct extension: Cars/Porsche/250x188.webp
Image does not have the correct extension: Cars/Porsche/2017-porsche-911-gt3-pdk-first-drive.webp
Image does not have the correct extension: Cars/Porsche/x96kiuu2.webp
Image does not have the correct extension: Cars/Porsche/1280x960.webp
Image does not have the correct extension: Cars/Porsche/08.jpg
Image does not have the correct extension: Cars/Porsche/vector3.svg
Image does not have the correct extension: Cars/Porsche/250x1883.webp
Image does not have the correct extension: Cars/Porsche/vector2.svg
Image does not have the correct extension: Cars/Porsche/vector6.svg
Image does not have the correct extension: Cars/Porsche/250x1882.webp
Image does not have the correct extension: Cars/Porsche/vector7.svg
Image does not have the correct extension: Cars/Porsche/2023-Porsche

NotADirectoryError: [Errno 20] Not a directory: 'Cars/.DS_Store'